In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge

from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, GridSearchCV

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

Читаем данные:

In [2]:
train = None
test = None
target = None

X_train = None,
X_test = None,
y_train = None,
y_test = None

def import_data():
    global train, test, target
    train = pd.read_csv('train_scaled.csv', index_col=0)
    test = pd.read_csv('test_scaled.csv', index_col=0)
    target = train['Price']

    train.drop(columns=['Price'], inplace=True)


In [3]:
def split():
    global X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.25, random_state=17)

In [4]:
import_data()
split()

In [5]:
results = None

def gradient_boositing_comp():
    global results, test, train
    trn = train.copy()
    ids = test['Id']
    tst = test.drop(columns=['Id'])
    
    gb_model = GradientBoostingRegressor(max_depth = 7, n_estimators= 100)
    gb_model.fit(trn, target)
    y_test_preds = gb_model.predict(tst)
    
    results = pd.DataFrame({'Id': ids, 'Price': y_test_preds})
    results.to_csv('MKremlev_predictions.csv', index=None)

gradient_boositing_comp()

Здесь пискал оптимальные параметры по лучшей модели:

Здесь меняя вариант предобработки, на наборе моделей оценивал эффект:

Предварительные выводы по влиянию факторов предобработки:
- масштабирование в текущем виде почти не влияет, немного ухудшает.
- сбросить в edge/median не сильно влияет если затрагивает небольшое число значений.
- удаление признака с большим числом пропусков Healthcare_1 вместо того чтобы заполнить пропуски ухудшает результат.
- если убрать корректировку этажей результаты значительно улучшаются.

Записал влияние ряда процедур предобработки, но не все:

base: 
1. linear_regression:  0.4729136872323022
2. linear_regular:     0.45689797534046417
3. random_forest:      0.6743797086260473
4. gradient_booster:   0.7036412199521744

убрал масштабирование:
1. linear_regression:  0.47291368723230187
2. linear_regular:     0.4729133895229596
3. random_forest:      0.6762857381967619
4. gradient_booster:   0.7006338247116466

вернул масштабирование, стал сбрасывать не в медиану, а в edge:
1. linear_regression:  0.4712899868532363
2. linear_regular:     0.45436059882286073
3. random_forest:      0.6762028696985951
4. gradient_booster:   0.7040286908956346

удалил Healthcare_1 вместо заполнения пропусков:
1. linear_regression:  0.4661754024914292
2. linear_regular:     0.4489348842399594
3. random_forest:      0.670451803968009
4. gradient_booster:   0.6931524421803156

убрал корректировку этажей:
1. linear_regression:  0.4767295391713775
2. linear_regular:     0.45852431805658866
3. random_forest:      0.6759433691222179
4. gradient_booster:   0.7149343456569763